# Collecting and preprocessing

## Importo las librerias necesarias para esta parte del proyecto

In [1]:
from datetime import datetime, timedelta
import time
import json
from collections import namedtuple
import pandas as pd
import matplotlib.pyplot as plt

## Datos extraidos de la web de la [AEMET](https://opendata.aemet.es/centrodedescargas/productosAEMET?), usanso el API key que me proporcionaron al registrarme

#### Extraigo los datos de 3 años, desde el 01-01-2018 hasta el 31-12-2020 y lo almaceno en un JSON
#### La estacion meteorologica que me ha proporcionado los datos, esta situada en Ciudad Universitaria

#### Voy a usar los siguientes datos proporcionados por la AEMET: 
- Fecha: `fecha`
- Temperatura Media: `tmed`
- Temperatura Minima: `tmin`
- Temperatura Maxima: `tmax`

In [2]:
ruta_archivo_json = 'datos.json'

with open(ruta_archivo_json) as archivo_json:
    datos_json = json.load(archivo_json)
    
df = pd.DataFrame(datos_json)
df = df.set_index('fecha')

#### Debido a que no tengo demasiado conocimiento sobre climatologia, hare como en el tutorial proporcionado y usare informacion de los 3 dias anteriores para la prediccion. La intencion es añadir variables al modelo

#### Para cada dia (fila), y cada medida (columna) que tengo, voy a buscar el valor de esa medida N dias antes y creare una columna nueva para esa medida mediante una funcion

In [3]:
#Funcion que hace lo explicitado anteriormente
def derive_nth_day_feature(df, feature, N):
    rows = df.shape[0]
    nth_prior_measurements = [None]*N + [df[feature][i-N] for i in range(N, rows)]
    col_name = "{}_{}".format(feature, N)
    df[col_name] = nth_prior_measurements

#### A continuacion, mediante un bucle, aplicare toda la funcion al dataframe principal
#### Con estos nuevos datos podre predecir mucho mejor el clima

In [4]:
features = ['tmed', 'prec', 'tmin', 'tmax']
for feature in features:
    if feature != 'date':
        for N in range(1, 4):
            derive_nth_day_feature(df, feature, N)

In [5]:
#Columnas resultantes
df.columns

Index(['indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin',
       'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha',
       'tmed_1', 'tmed_2', 'tmed_3', 'prec_1', 'prec_2', 'prec_3', 'tmin_1',
       'tmin_2', 'tmin_3', 'tmax_1', 'tmax_2', 'tmax_3'],
      dtype='object')

## Data cleaning

#### Elimino las columnas que no voy a necesitar, de esta manera trabajare con menos datos

In [6]:
df = df.drop(['prec','indicativo','nombre','provincia','altitud','horatmin','horatmax','dir','velmedia','racha','horaracha'],axis=1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1089 entries, 2018-01-01 to 2020-12-31
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tmed    938 non-null    object
 1   tmin    938 non-null    object
 2   tmax    938 non-null    object
 3   tmed_1  937 non-null    object
 4   tmed_2  936 non-null    object
 5   tmed_3  935 non-null    object
 6   prec_1  1084 non-null   object
 7   prec_2  1083 non-null   object
 8   prec_3  1082 non-null   object
 9   tmin_1  937 non-null    object
 10  tmin_2  936 non-null    object
 11  tmin_3  935 non-null    object
 12  tmax_1  937 non-null    object
 13  tmax_2  936 non-null    object
 14  tmax_3  935 non-null    object
dtypes: object(15)
memory usage: 136.1+ KB


#### Como se puede observar, lo siguiente a realizar es dar el formato adecuado a las columnas del dataframe, usare para ello el metodo `to_numeric` de pandas

In [8]:
#Lo primero es sustituir las comas por puntos
df = (df.replace(',','.', regex=True).astype(float))
#Ahora convierto a un tipo de dato numerico
df = df.apply(pd.to_numeric, errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1089 entries, 2018-01-01 to 2020-12-31
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tmed    938 non-null    float64
 1   tmin    938 non-null    float64
 2   tmax    938 non-null    float64
 3   tmed_1  937 non-null    float64
 4   tmed_2  936 non-null    float64
 5   tmed_3  935 non-null    float64
 6   prec_1  1084 non-null   float64
 7   prec_2  1083 non-null   float64
 8   prec_3  1082 non-null   float64
 9   tmin_1  937 non-null    float64
 10  tmin_2  936 non-null    float64
 11  tmin_3  935 non-null    float64
 12  tmax_1  937 non-null    float64
 13  tmax_2  936 non-null    float64
 14  tmax_3  935 non-null    float64
dtypes: float64(15)
memory usage: 136.1+ KB


#### Ahora que tengo el dataframe con el formato adecuado, tengo que utilizar reglas estadisticas para ver si hay casos extremos. 

#### Usare para ello la funcion `describe()` que me proporcionara varios datos estadisticos como la media, los percentiles etc...

#### Esta informacion es muy util para saber la distribucion de los datos

#### Añadire otra columna (outliers) que indicara si en esa columna hay outliers, es decir, valores extremos.

In [9]:
# Llamo a decribe() para ver el df y lo transpongo para mejorar su visualizacion
spread = df.describe().T

#Precalculo el rango intercuartilico ya que me sera util para los proximos calculos
IQR = spread['75%'] - spread['25%']

# Creo la columna de outliers
spread['outliers'] = (spread['min']<(spread['25%']-(3*IQR)))|(spread['max'] > (spread['75%']+3*IQR))

# Muestro solamente las medidas que tiene outliers
spread.loc[spread.outliers,]

,count,mean,std,min,25%,50%,75%,max,outliers
prec_1,1084.0,1.194926,4.034098,0.0,0.0,0.0,0.1,43.8,True
prec_2,1083.0,1.196030,4.035798,0.0,0.0,0.0,0.1,43.8,True
prec_3,1082.0,1.197135,4.037500,0.0,0.0,0.0,0.1,43.8,True


#### Es muy complicado evaluar el impacto potencial de estos outliers

#### Por un lado, puede que añadan datos erroneos(espuria) que tengan un impacto significativo en el modelo

#### En cambio, me pueden ser tremendamente utiles para predecir algunos resultados que ocurren bajo unas determinadas circunstancias

### En este caso, al tratarse de la precipitacion, es muy comprensible la presencia de outliers. Debido a que los dias con 0% de precipitacion son muy frecuentes. Por este motivo, mantendre los datos.

### Finalmente, debo decidir que hacer con los datos que faltan, que estan representados como NaNs. Ya que la ausencia de datos, puede ser un problema para los metodos de machine learning.

#### Muchos de ellos se deben a las primeras filas, ya que no hay datos anteriores. Elimino esas filas.

#### Viendo el resultado de `df.info()` se observa que en todas las columnas faltan unos 140 datos, excepto en las precipitaciones que estan practicamente completas.

### Tengo dos opciones, eliminar esas filas, o rellenarlas haciendo interpolacion.
#### Debido a que la interpolacion no sera facil, he decidido eliminarlas


In [10]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 2018-01-04 to 2020-12-31
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tmed    912 non-null    float64
 1   tmin    912 non-null    float64
 2   tmax    912 non-null    float64
 3   tmed_1  912 non-null    float64
 4   tmed_2  912 non-null    float64
 5   tmed_3  912 non-null    float64
 6   prec_1  912 non-null    float64
 7   prec_2  912 non-null    float64
 8   prec_3  912 non-null    float64
 9   tmin_1  912 non-null    float64
 10  tmin_2  912 non-null    float64
 11  tmin_3  912 non-null    float64
 12  tmax_1  912 non-null    float64
 13  tmax_2  912 non-null    float64
 14  tmax_3  912 non-null    float64
dtypes: float64(15)
memory usage: 114.0+ KB


### Finalmente, tengo 912 columnas, en lugar de las 1089 iniciales, no es excesivo el corte que he hecho

### -----------------------EXPORTACION DE DATOS----------------------

In [11]:
df.to_csv (r'end-part1.csv', header=True)

In [12]:
import pickle
with open('end-part1_df.pkl', 'wb') as fp:
    pickle.dump(df, fp)